## RNN

# 下面的代码来对mnist数据集使用RNN来进行分类

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn
#载入数据
minist = input_data.read_data_sets("./mnist/", one_hot=True)
#输入图片是28*28
#一行是28
n_input = 28
#一共28行
max_time = 28
#隐层单元
lstm_size = 100
#一共10个分类
n_classes = 10
#每批次50个样本
batch_size = 50
#一共多少个批次
n_batch  = minist.train.num_examples//batch_size
 
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
 
#初始化权值
w = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
#初始化偏置值
b = tf.Variable(tf.constant(0.1,shape=[n_classes]))
 
#定义RNN网络
def RNN(x,weights,biases):
    #inputs = [batch_size,max_time,n_inputs]
    inputs = tf.reshape(x,[-1,max_time,n_input])
    #定义LSTM基本CELL
    lstm_cell = rnn.BasicLSTMCell(lstm_size)
    #final_state[0]是cell_state,  final_state[1]是hidden_state
    out_puts,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    result = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
    return result
 
#计算RNN的返回结果
prediction = RNN(x,w,b)
#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔列表中
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
# 保存训练好的模型和参数
saver = tf.train.Saver()
init_op = tf.global_variables_initializer()    # 变量初始化
with tf.Session() as sess:
    sess.run(init_op)
    for epoch in range(1, 6, 1):
        for batch in range(n_batch):
            batch_xs,batch_ys = minist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        acc = sess.run(accuracy,feed_dict={x:minist.test.images,y:minist.test.labels})
        print('Iterator : '+str(epoch)+' accuracy : '+str(acc))
    saver.save(sess, "./RNN_Save/model.ckpt")

H:\anaconda_env\anaconda_install\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iterator : 1 accuracy : 0.7228
Iterator : 2 accuracy : 0.8503
Iterator : 3 accuracy : 0.8899
Iterator : 4 accuracy : 0.9027
Iterator : 5 accuracy : 0.9183


# 重新载入模型参数

In [4]:
#用于保存训练好的模型和参数
saver = tf.train.Saver()
 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "./RNN_Save/model.ckpt")
    for epoch in range(1, 3, 1):
        for batch in range(n_batch):
            batch_xs,batch_ys = minist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        acc = sess.run(accuracy,feed_dict={x:minist.test.images,y:minist.test.labels})
        print('Iterator : '+str(epoch)+' accuracy : '+str(acc))
    saver.save(sess, "./RNN_Save/model.ckpt")

INFO:tensorflow:Restoring parameters from ./RNN_Save/model.ckpt
Iterator : 1 accuracy : 0.9388
Iterator : 2 accuracy : 0.9375
